This notebook is aimed at identifying unphased regions of the priamry contigs based on short read illumina mapping data. Parts of it are really slow.


This notebook was only designed for the purpose of analyzing the Pst-104E genome and now was adapted to the DK09-11 genome. No gurantees it works in any other situtation. It will have spelling errors due to the lack of autocorrection.

### Coverage analysis script
The original idea of the script is to simply use the coverage dataframe generated by 
samtools depth -aa ${x} > ${x}.aa.cov and pull out 1x homozygous and unique coverage areas. This is fr now done with the 'DK_0911_v03_p_ctg.bwamem.PRI_NTKN_DK0911.sam.sorted.bam.aa.cov' files.
Think about use DeepTools in future when comparing mulitple mapping experiments.

Future version of this script should considered tilling sliding windows instead of per base callculation. The choosen cut off seem reasonable based on the observed values of coverage. This may not be the case for all genomes though. Hence a sliding window approach maybe better.

#### preserved comments and ideas from the origianl version
do a groupby and the mean on this. get ride of the high coverage contigs. calcutate standard statistics. Do a rolling window on each group and convert to a gff file kind of format.
also look at the coverage on p only and calcualte coverage there in the same way this should be the diploid coverage really (kind off). 
Think about combining the two dataframes for pcontigs and do a covariation analysis on the sliding windows look for areas where there is no change for p vs ph mapping and high coverage in both cases. This should be a homozygous region.

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn
import matplotlib
from Bio import SeqIO, SeqUtils
import os

In [3]:
#Define the PATH
BASE_AA_PATH = '/home/benjamin/genome_assembly/Warrior'
BASE_A_PATH = '/home/benjamin/genome_assembly/Warrior/genome_v04'
#for now use the previous mapping that still included high coverage regions
COV_IN_PATH = '/home/benjamin/genome_assembly/Warrior/SRM'
BAM_IN_PATH = '/home/benjamin/genome_assembly/Warrior/SRM'
#apply analysis restricted to final assembly Pst_104E_v12
COV_OUT_PATH = os.path.join(BASE_AA_PATH, 'COV')
if not os.path.isdir(COV_OUT_PATH):
    os.mkdir(COV_OUT_PATH)
pwh_list_path = os.path.join(BASE_A_PATH, 'DK_0911_v04_pwh.list')
pwoh_list_path = os.path.join(BASE_A_PATH, 'DK_0911_v04_pwoh.list')

In [4]:
input_genome = 'DK0911_v03'
coverage_file_suffix = 'bwamem.PRI_NTKN_DK0911.sam.sorted.bam.aa.cov'
output_genome = 'DK0911_v04'

In [5]:
#get all the cov files with this coverage_file_suffix. Expacts to get all three p, h and ph mapping
cov_files = [os.path.join(COV_IN_PATH, x) for x in os.listdir(COV_IN_PATH) if x.endswith(coverage_file_suffix)]

In [ ]:
cov_header = ["contig", "position", 'coverage']

In [ ]:
ph_cov = pd.read_csv([y for y in cov_files if 'ph_ctg' in y][0], sep='\t', header=None, names=cov_header)
print('Read in following file as ph_coverage produced with samtools depth -aa feature: %s' %[y for y in cov_files if 'ph_ctg' in y][0])

In [ ]:
p_cov = pd.read_csv([y for y in cov_files if 'p_ctg' in y][0], sep='\t', header=None, names=cov_header)
print('Read in following file as p_coverage produced with samtools depth -aa feature: %s' %[y for y in cov_files if 'p_ctg' in y][0])

In [ ]:
h_cov = pd.read_csv([y for y in cov_files if '_h_ctg' in y][0], sep='\t', header=None, names=cov_header)
print('Read in following file as h_coverage produced with samtools depth -aa feature: %s' %[y for y in cov_files if '_h_ctg' in y][0])

In [ ]:
#summarize the mean coverage by contigs for p_contigs when mapping against p_contigs only
mean_cov_per_contig = p_cov.groupby('contig').mean()

mean_cov_per_contig['coverage'].plot.box()

overall_mean = p_cov['coverage'].mean()

overall_std = p_cov['coverage'].std()

print("The mean overall coverage is %.2f and the std is %.2f for p" % (overall_mean, overall_std))

In [ ]:
per_contig_mean_p = mean_cov_per_contig['coverage'].mean()
per_contig_std_p = mean_cov_per_contig['coverage'].std()
print("The per contig mean on p is %.2f and the std is %.2f for p" % (per_contig_mean_p , per_contig_std_p))

In [ ]:
#summarize the mean coverage by contigs for all contigs when mapping against p and h contigs. Plot only coverage plot for p_contigs
mean_cov_per_contig_ph = ph_cov.groupby('contig').mean()
mean_cov_per_contig_ph['contig'] = mean_cov_per_contig_ph.index
mean_cov_per_contig_ph['coverage'].plot.box(sym ='r+')
overall_mean_ph = ph_cov['coverage'].mean()

overall_std_ph = ph_cov['coverage'].std()

print("The mean overall per base coverage is %.2f and the std is %.2f for ph mapping" % (overall_mean_ph, overall_std_ph))

In [ ]:
#summarize the mean coverage by contigs for all contigs when mapping against p and h contigs. Plot only coverage plot for p_contigs
mean_cov_per_contig_ph_p = mean_cov_per_contig_ph[mean_cov_per_contig_ph['contig'].str.contains('pcontig')]
mean_cov_per_contig_ph_p['coverage'].plot.box(sym ='r+')
overall_mean_ph_p = mean_cov_per_contig_ph_p['coverage'].mean()

overall_std_ph_p = mean_cov_per_contig_ph_p['coverage'].std()

print("The mean per contig coverage on p is %.2f and the std is %.2f for ph mapping" % (overall_mean_ph_p, overall_std_ph_p))

In [ ]:
mean_cov_per_contig_ph[mean_cov_per_contig_ph['contig'].str.contains('hcontig')]['coverage'].plot.box(sym ='r+')

In [ ]:
#summarize the mean coverage by contigs for h contigs when mapping against h contigs. Plot only coverage plot for h_contigs
mean_cov_per_contig_h = h_cov.groupby('contig').mean()
mean_cov_per_contig_h['contig'] = mean_cov_per_contig_h.index
mean_cov_per_contig_h['coverage'].plot.box(sym ='r+')

overall_mean_h = h_cov['coverage'].mean()

overall_std_h = h_cov['coverage'].std()

print("The mean per base coverage is %.2f and the std is %.2f for ph mapping" % (overall_mean_h, overall_std_h))

In [ ]:
mean_cov_per_contig.columns

In [ ]:
#these are the primary contigs with a coverage greater than 50
pcongtig_greater_50 = list(mean_cov_per_contig[mean_cov_per_contig.coverage > 50 ].index.unique())

In [ ]:
pcongtig_greater_50

In [ ]:
congtig_ph_greater_50 = list(mean_cov_per_contig_ph[mean_cov_per_contig_ph.coverage > 50 ].index.unique())

In [ ]:
congtig_ph_greater_50

Look at contigs 52, 101, and 106 if this is something particular

In [ ]:
hcongtig_greater_50 = list(mean_cov_per_contig_h[mean_cov_per_contig_h.coverage > 50 ].index.unique())

In [ ]:
hcongtig_greater_50

In [ ]:
{x[:-4] for x in hcongtig_greater_50}

In [ ]:
#get a ph coverage dataframe that does not included the pcontigs greater 50x coverage
ph_cov_smaller_50 = ph_cov[~ph_cov['contig'].isin(pcongtig_greater_50)]

In [ ]:
#claculate the mean contig coverage
mean_cov_ph_contig_smaller_50 = ph_cov_smaller_50.groupby('contig').mean()

In [ ]:
#fix this up by removing hcontigs associated with pcontigs that have greater 50x coverage
mean_cov_ph_contig_smaller_50 = mean_cov_ph_contig_smaller_50[~mean_cov_ph_contig_smaller_50.index.str[:-4].isin(\
                              [x.replace('pc', 'hc') for x in pcongtig_greater_50])]

In [ ]:
#get the list of p contigs with haplotig [pwh] and without haplotig [pwoh]
pwh_list = pd.read_csv(pwh_list_path, sep='\t', header=None)[0].tolist()
pwoh_list = pd.read_csv(pwoh_list_path, sep='\t', header=None)[0].tolist()

In [ ]:
#mean_cov_contig_s2000_ph_pwh represents all contigs that are below 2000x coverage and are pwh contigs
mean_cov_ph_contig_smaller_50_pwh = mean_cov_ph_contig_smaller_50[mean_cov_ph_contig_smaller_50.index.isin(pwh_list)]
mean_cov_ph_contig_smaller_50_pwh['coverage'].plot.box()
plt.title('mean_cov_contig_s50_ph_pwh')
mean_s50_ph_pwh = mean_cov_ph_contig_smaller_50_pwh['coverage'].mean()
std_s50_ph_pwh = mean_cov_ph_contig_smaller_50_pwh['coverage'].std()
print("The mean per contig coverage for s50 contigs of pwh contigs while ph mapping is %.2f and the std is %.2f" % (mean_s50_ph_pwh, std_s50_ph_pwh))

In [ ]:
#mean_cov_contig_s2000_ph_pwh represents all contigs that are below 2000x coverage and are pwh contigs
mean_cov_ph_contig_smaller_50_pwoh = mean_cov_ph_contig_smaller_50[mean_cov_ph_contig_smaller_50.index.isin(pwoh_list)]
mean_cov_ph_contig_smaller_50_pwoh['coverage'].plot.box()
plt.title('mean_cov_contig_s50_ph_pwoh')
mean_s50_ph_pwoh = mean_cov_ph_contig_smaller_50_pwoh['coverage'].mean()
std_s50_ph_pwoh = mean_cov_ph_contig_smaller_50_pwoh['coverage'].std()
print("The mean per contig coverage for s50 contigs of pwoh contigs while ph mapping is %.2f and the std is %.2f" % (mean_s50_ph_pwoh, std_s50_ph_pwoh))

In [ ]:
#mean_cov_contig_s2000_ph_pwh represents all contigs that are below 2000x coverage and are pwh contigs
mean_cov_ph_contig_smaller_50_h = mean_cov_ph_contig_smaller_50[mean_cov_ph_contig_smaller_50.index.str.startswith('hcontig')]
mean_cov_ph_contig_smaller_50_h['coverage'].plot.box()
plt.title('mean_cov_contig_s50_ph_h')
mean_s50_ph_h = mean_cov_ph_contig_smaller_50_h['coverage'].mean()
std_s50_ph_h = mean_cov_ph_contig_smaller_50_h['coverage'].std()
print("The mean per contig coverage for s50 contigs of h contigs while ph mapping is %.2f and the std is %.2f" % (mean_s50_ph_h, std_s50_ph_h))

In [ ]:
mean_cov_ph_contig_smaller_50_p = mean_cov_ph_contig_smaller_50[mean_cov_ph_contig_smaller_50.index.str.startswith('pcon')]

In [ ]:
mean_cov_ph_contig_smaller_50_p['coverage'].plot.box()
plt.title('mean_cov_contig_s50_ph_p')
mean_s50_ph_p = mean_cov_ph_contig_smaller_50_p['coverage'].mean()
std_s50_ph_p = mean_cov_ph_contig_smaller_50_p['coverage'].std()
print("The mean per contig coverage for s50 contigs of p contigs while ph mapping is %.2f and the std is %.2f" % (mean_s50_ph_p, std_s50_ph_p))

In [ ]:
#think about what thresholds to pick in the long run.
threshold_up_ph_p = mean_s50_ph_p + 2*std_s50_ph_p 
threshold_down_ph_p = mean_s50_ph_p - 2*std_s50_ph_p 

#threshold_up = mean_s2000 + 2*std_s2000
#threshold_down = mean_s2000 - 2*std_s2000

In [ ]:
#potnetial fully homozygous contigs
mean_cov_per_contig_ph_pwoh = mean_cov_per_contig_ph[mean_cov_per_contig_ph.index.isin(pwoh_list)]
mean_cov_per_contig_ph_pwoh[mean_cov_per_contig_ph_pwoh.coverage > threshold_up_ph_p]['coverage']

In [ ]:
mean_file_name = os.path.join(COV_OUT_PATH, '%s_mean_cov.txt' % output_genome)

In [ ]:
with open(mean_file_name,'w') as outfn:
    print('mean_s50_ph_p: %f' % mean_s50_ph_p,file=outfn)
    print('std_s50_ph_p: %f' % std_s50_ph_p, file=outfn)

In [ ]:
!cat {mean_file_name}

In [ ]:
ph_cov.head()

In [ ]:
print(ph_cov[ph_cov.contig.str.startswith('pcont')]['coverage'].mean())
print(ph_cov[ph_cov.contig.str.startswith('pcont')]['coverage'].std())

In [ ]:
ph_cov_smaller_50[ph_cov_smaller_50.contig.str.startswith('pcont')].head()

In [ ]:
#potnetial fully homozygous contigs if we are not considering the presence of haplotigs. 
#some of these haplotigs may also be collapsed compared to there primary contigs.
mean_cov_per_contig_ph[mean_cov_per_contig_ph.coverage > threshold_up_ph_p]['coverage']

#### Looking a bit into the nucmer plots of these contigs for visualization
* the hcontigs_000_xxx are fairly small sequences and maybe repeats.


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%run DK_0911_coverage_analysis_submission_part2_parallel.ipynb